In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load the trained model
model = load_model('best_model.h5')

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [3]:
# Label encoder for decoding predictions
with open('data.pickle', 'rb') as f:
    dataset = pickle.load(f)  # Pass the file object `f` to pickle.load()

label_encoder = LabelEncoder()
label_encoder.fit(dataset['labels'])

LabelEncoder()

In [4]:
import cv2
import numpy as np
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont

# Create a dictionary to map numerical labels to Sinhala letters
labels_dict = {0: 'අ', 1: 'ආ', 2: 'ඇ', 3: 'ඉ',  4: 'ඊ',  5: 'උ',  6: 'ඌ',  7: 'එ',  8: 'ඒ',  9: 'ක්', }

# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Access the camera
cap = cv2.VideoCapture(0)

# Load a font that supports Sinhala characters (make sure the font file is available)
try:
    font_path = "NotoSansSinhala-VariableFont_wdth,wght.ttf"  # Provide correct path to a font that supports Sinhala
    font = ImageFont.truetype(font_path, 30)
except IOError:
    print("Font file not found. Defaulting to a basic font.")
    font = ImageFont.load_default()  # Use default font if the custom font is not found

# Initialize the hand landmark detection model
hands = mp_hands.Hands()

print("Press 'Esc' to quit.")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame.")
        break

    # Convert the frame to RGB (for PIL)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)

    # Process the image using MediaPipe
    results = hands.process(img_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks with custom color
            for landmark in hand_landmarks.landmark:
                # Convert normalized coordinates to pixel values
                h, w, _ = frame.shape
                x = int(landmark.x * w)
                y = int(landmark.y * h)

                # Draw circle for each landmark with custom color (e.g., red)
                cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # (0, 0, 255) is red color in BGR

            # Draw the hand connections (lines between landmarks)
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract hand landmarks
            data_aux = []
            x_ = []
            y_ = []
            for lm in hand_landmarks.landmark:
                x_.append(lm.x)
                y_.append(lm.y)

            # Normalize landmarks
            for lm in hand_landmarks.landmark:
                data_aux.append((lm.x - min(x_)) / (max(x_) - min(x_)))
                data_aux.append((lm.y - min(y_)) / (max(y_) - min(y_)))

            # Predict the letter
            data_aux = np.array(data_aux).reshape(1, -1, 1)  # Reshape to 3D
            prediction = model.predict(data_aux)
            predicted_label_index = np.argmax(prediction)
            predicted_letter = labels_dict[predicted_label_index]  # Map to Sinhala letter

            # Convert frame to PIL image again if necessary
            draw = ImageDraw.Draw(pil_img)
            draw.text((50, 50), f'Letter: {predicted_letter}', font=font, fill=(255, 0, 0))

    # Convert PIL image back to OpenCV format
    frame = np.array(pil_img)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Display the frame with hand landmarks and the predicted letter
    cv2.imshow('Live Prediction', frame)

    # Exit on 'Esc' key
    if cv2.waitKey(10) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Press 'Esc' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 632ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 